In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from iso8601 import parse_date

import pprint
pp = pprint.PrettyPrinter().pprint

import sys

mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['font.size'] = 18

In [3]:
from satbazaar import db
from satbazaar import schedulers
from satbazaar import client

In [4]:
# script 'get-stations.py' creates this file
stations = db.load_stations()

# script 'get-satellites.py' creates both files
satellites = db.load_satellites()

In [5]:
gs = stations['KB9JHU']

# script 'generate-allpasses.py' generates the database of sat-gs pass predictions
passes = db.getpasses(gs=gs['name'], start='2018-07-11', end='2018-07-11 04:00')
duration = (passes.end() - passes.begin()).total_seconds()
print('duration:        {:.3f}'.format(duration))

maxtree = passes.copy()
maxtree.merge_overlaps()
total = 0.0
for i in maxtree:
    total += (i.end - i.begin).total_seconds()
print('total pass time: {:.3f}'.format(total))

duration:        14317.008
total pass time: 14061.390


In [6]:
def do_plot():
    fig, ax = plt.subplots(figsize=(8, 4))

    ax.bar(names, busys)

    labels = ax.get_xticklabels()
    plt.setp(labels, rotation=20)
    
    print('{:30s} {:>5s}  {:>5s}'.format('Scheduler', 'busy', 'eff'))
    for n, b in zip(names, busys):
        print('{:30s} {:5.1f}  {:5.1f}'.format(n, 100*b/duration, 100*b/total))

In [8]:
def plot_result(c, name):
    # collect satellite names
    sats = set([p.data['job']['tle0'] for p in c.calendar])
    sats = list(sats)
    sats.sort()
    sats.reverse()
    #index is used as y-offset


    # sats simultaneously overhead
    start = c.calendar.begin()  #parse_date('2018-07-11')
    end = c.calendar.end()  #parse_date('2018-07-11 04:00')

    total_minutes = (end - start).total_seconds()/60
    steps = [start + timedelta(minutes=x) for x in range(int(total_minutes))]
    num = []
    for t in steps:
        passes = c.calendar[t]
        num.append(len(passes))

        
    #plt.close('all')
    fig = plt.figure(figsize=(15, 6))
    ax = plt.subplot()

    ax.xaxis_date()
    ax.set_ylim((0.9, 2.0))

    from itertools import cycle, islice
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']

    for p in c.calendar:
        sat = p.data['job']['tle0']
        idx = sats.index(sat)
        ax.plot(
            (p.begin, p.end),
            2*[1],
            '-|',
            color=colors[idx%len(colors)],
        )
        ax.text(
            p.begin + (p.end - p.begin)/2,
            1.0,
            ' ' + sat,
            rotation=90,
            ha='center',
            va='bottom',
            fontsize=18,
        )

    #ax.set_yticks(range(len(sats)))
    #ax.set_yticklabels(sats)
    ax.set_title('{}\n{}'.format(name, c))
    ax.grid(True, axis='x')

    plt.tight_layout()
    fig.autofmt_xdate()

    plt.savefig('kb9jhu-sched.png', dpi=300, bbox_inches='tight')

In [9]:
options = (
    #schedulers.RandomScheduler,
    schedulers.FirstScheduler,
    schedulers.LastScheduler,
    schedulers.DurationScheduler,
    #schedulers.EndStartScheduler,
    schedulers.OwnerPreferenceScheduler,
)

priority = {
    'KB9JHU':(
        39092,  # UNIBRITE
        28654,  # NOAA 18
        40903,  # XW-2A
    ),
}


names = []
busys = []
for sch in options:
    c = client.YesClient(gs)
    clients={gs['name']:c}

    if sch == schedulers.OwnerPreferenceScheduler:
        s = sch(clients, satellites, priority=priority)
    else:
        s = sch(clients, satellites)

    s(passes)
    
    plot_result(c, sch.__name__)

    busys.append(c.busy_time())
    names.append(sch.__name__)




do_plot()


AttributeError: 'IntervalTree' object has no attribute 'search'